In [1]:
import json
from pprint import pprint
import pandas as pd
import requests


def prepdata(liquidations):    
    rows = []

    for x in liquidations:
        collat_amount = int(x["collateralAmount"]) / 10**int(x["collateralReserve"]["decimals"])
        borrow_amount = int(x["principalAmount"]) / 10**int(x["principalReserve"]["decimals"])

        collat_value_usd = float(x["collateralAssetPriceUSD"])*collat_amount
        borrow_value_usd = float(x["borrowAssetPriceUSD"])*borrow_amount

        rows.append(
            {
                "txnHash": x["txHash"],
                "blockNumber": int(x["id"].split(":")[0]),
                "blockPosition": int(x["id"].split(":")[1]),

                "user": x["user"]["id"],
                "liquidator": x["liquidator"],

                "collateralAsset": x["collateralReserve"]["symbol"],
                "borrowAsset": x["principalReserve"]["symbol"],
                "collateralAmount": collat_amount,
                "borrowAmount": borrow_amount,
                "collateralAssetPriceUSD": float(x["collateralAssetPriceUSD"]),
                "borrowAssetPriceUSD": float(x["borrowAssetPriceUSD"]),
                "collateralValueUSD": collat_value_usd,
                "borrowValueUSD": borrow_value_usd,

                "profitUpperBoundUSD": collat_value_usd-borrow_value_usd,
            }
        )
    
    df = pd.DataFrame(rows)
    df = df.sort_values(by=["blockNumber", "blockPosition"], ascending=True) # sorts the transactions by occurrence within the block
    return df

def getliquidations(cutoff_timestamp=1669224247, subgraph_url="https://api.thegraph.com/subgraphs/name/aave/protocol-v2"):
    """
    Get all of the Aave liquidations, back
    to a specific time `cutoff_timestamp`.
    
    cutoff_timestamp: int
    """
    url = subgraph_url
    all_liquidations = []
    sup_timestamp = 1671816247 # 10**15 # greates that the timestamp can be, on each round of pagination
    
    for i in range(1_001):
        print("i:", i)
        
        query = '''
        {
            liquidationCalls(
                first: 1000, 
                orderBy: timestamp, 
                orderDirection: desc,
                where: {
                    timestamp_lte: %d,
                    #timestamp_gte: %d
                }
            ) {
                id
                txHash
                timestamp

                user {id}
                liquidator
                pool {id}


                collateralAmount
                principalAmount
                
                collateralReserve {decimals, symbol}
                
                principalReserve {decimals, symbol}
                
                collateralAssetPriceUSD
                borrowAssetPriceUSD
            }
        }
        ''' % (
            sup_timestamp,
            cutoff_timestamp
        )

        result = json.loads(requests.post(
            url,
            json={"query": query}
        ).content)

        if ("data" not in result) or (len(result["data"]["liquidationCalls"]) == 0):
            pprint(result)
            print("breaking due to error in response data")
            break

        liquidations = result["data"]["liquidationCalls"]

        # linear-time search for the last timestamp; can be optimized to O(log(n)), but not worth it rn
        j = len(liquidations)
        while (j>0) and (int(liquidations[j-1]["timestamp"]) < cutoff_timestamp):
            j -= 1
        
        if j == 0:
            print("Something went wrong, since all of the liquidations occurred before the cutoff. Stopping now.")
            break
        

        if sup_timestamp == int(liquidations[j-1]["timestamp"]):
            # note, this could be problematic if we had >1,000 liquidations that all had the same timestamp
            print("done, since we have the same sup_timestamp as before")
            break
        else:
            sup_timestamp = int(liquidations[j-1]["timestamp"]) # set the maximum time
            # print("sup timestamp", sup_timestamp, "\n")

        # concatenate all liquidations that have timestamp >= cutoff_timestamp
        all_liquidations += liquidations[:j]
        if j != len(liquidations):
            print("done, since the final liquidation is in the interior of the result batch")
            break

        # break
                    
    pprint(all_liquidations[-1])
            
    return prepdata(all_liquidations)

In [10]:
import datetime as dt
aave_liquidations_ = getliquidations(
    int(dt.datetime(2022, 1, 1).timestamp()),
); aave_liquidations_

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
done, since the final liquidation is in the interior of the result batch
{'borrowAssetPriceUSD': '1.001593261522784862307876507014685',
 'collateralAmount': '461896212893412717',
 'collateralAssetPriceUSD': '3701.838215730009042169811095635005',
 'collateralReserve': {'decimals': 18, 'symbol': 'WETH'},
 'id': '13919368:72:0xa7a05399861e5d27df713654601283321629c4062734c160e22f1bd8d04ef45c:99:99',
 'liquidator': '0x3909336de913344701c6f096502d26208210b39f',
 'pool': {'id': '0xb53c1a33016b2dc2ff3653530bff1848a515c8c5'},
 'principalAmount': '1625852499',
 'principalReserve': {'decimals': 6, 'symbol': 'USDC'},
 'timestamp': 1641037458,
 'txHash': '0xa7a05399861e5d27df713654601283321629c4062734c160e22f1bd8d04ef45c',
 'user': {'id': '0x96954e2cc59046bb181a8b17919aab14e5264f70'}}


,txnHash,blockNumber,blockPosition,user,liquidator,collateralAsset,borrowAsset,collateralAmount,borrowAmount,collateralAssetPriceUSD,borrowAssetPriceUSD,collateralValueUSD,borrowValueUSD,profitUpperBoundUSD
19274,0xa7a05399861e5d27df713654601283321629c4062734...,13919368,72,0x96954e2cc59046bb181a8b17919aab14e5264f70,0x3909336de913344701c6f096502d26208210b39f,WETH,USDC,0.461896,1625.852499,3701.838216,1.001593,1709.865053,1628.442907,81.422145
19273,0x367f5f4cb77aaa3fb434d92faa2e95b7c06a1b347c36...,13919383,35,0x735415e648b0766bce64984b7fe0c5414d1053fe,0x3909336de913344701c6f096502d26208210b39f,WETH,USDT,0.504831,1787.590330,3701.838216,0.995649,1868.802799,1779.812190,88.990609
19272,0xd9ef157cf44f884dd347d49857fce6701f5f49ec44d5...,13919490,189,0x2f33f4a63b1928a67823df04897a133dba9c4f3b,0x3909336de913344701c6f096502d26208210b39f,WETH,USDT,0.443867,1571.717117,3702.140815,0.995730,1643.256509,1565.006199,78.250310
19271,0x9630822097eaf011583e30c4bc617222c07af85de75a...,13921935,90,0xfaf3af4f551f76af4cde17c3d3708c4f3a69d21e,0x3909336de913344701c6f096502d26208210b39f,WETH,YFI,1.396658,0.131580,3762.840000,38038.801351,5255.400539,5005.143370,250.257169
19270,0xab85193d071450cd95d68e9755131325bc27d5305489...,13929582,117,0xfaf3af4f551f76af4cde17c3d3708c4f3a69d21e,0x3909336de913344701c6f096502d26208210b39f,WETH,BAL,0.937014,175.359917,3814.803616,19.413265,3574.523933,3404.308507,170.215425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,0xf518fc2a8bab872919d25d7760957704e73f9731f2a7...,16235704,1,0x217dbf950c50bc2819a899146716583bc605abfa,0xb6569481dccddd527c2b0e8ba32f494e52224ca1,WETH,WETH,0.441978,0.420931,1208.814004,1208.814004,534.268724,508.827357,25.441368
3,0xbea7e992cb3be25fa340cd2e777fcaa6e14777d4049c...,16237474,3,0xffef69efa7f65e233252fdde813ba9f5032e2964,0xb6569481dccddd527c2b0e8ba32f494e52224ca1,WETH,SUSD,0.178253,205.396036,1214.417700,1.003745,216.473478,206.165217,10.308261
2,0xe0f1b5ba5b997a413ceff6f96d38c7df349cabf7c8a2...,16241016,1,0xd66ba9b7d332ed5f399ca907bab7686292b14cdf,0xae14d6f43eeb51e66cd1e5ca94269ecaeeddfb04,LINK,USDT,4020.235529,22233.315792,5.882085,0.994019,23647.368031,22100.343954,1547.024077
1,0xc26fb38db9e50752ef15b85de902415eaf6ca5577ae4...,16241057,3,0x828d6384759cb80da04f02b4490844ed34dd652b,0x6759674c937badbed743d56e5be86301968ea662,LINK,USDT,239.685368,1312.079394,5.850762,0.998872,1402.341978,1310.599979,91.741999


,txnHash,blockNumber,blockPosition,user,liquidator,collateralAsset,borrowAsset,collateralAmount,borrowAmount,collateralAssetPriceUSD,borrowAssetPriceUSD,collateralValueUSD,borrowValueUSD,profitUpperBoundUSD
19274,0xa7a05399861e5d27df713654601283321629c4062734...,13919368,72,0x96954e2cc59046bb181a8b17919aab14e5264f70,0x3909336de913344701c6f096502d26208210b39f,WETH,USDC,0.461896,1625.852499,3701.838216,1.001593,1709.865053,1628.442907,81.422145
19259,0x9f60c3ac43053af8bc7e746cdb4ada61351ea41cf4e7...,13947499,256,0x16c05424094e9d1374d3f6d707da7e91e933d467,0x3909336de913344701c6f096502d26208210b39f,WETH,USDC,4.105166,14251.453062,3648.713890,1.000972,14978.574383,14265.308937,713.265447
19253,0x16a22c8e49cf39b8f1764f30dea8cc7866b66941a105...,13947699,2,0x7b51647756ecbfcf3c028ae85e7e2eb88eaf718e,0x3909336de913344701c6f096502d26208210b39f,WETH,USDC,122.924134,422480.368437,3609.706873,1.000261,443720.093043,422590.564803,21129.528240
19254,0x38976b2ef5f33a896e2ac108e009d7bd1977729eef63...,13947699,19,0x40bbcf14f31cd276ad432bc485027bf226ae3b4a,0xb2b3d5b6215d4fb23bf8dd642d385c4b44aadb2a,WETH,USDC,13.097230,45014.128011,3609.706873,1.000261,47277.162589,45025.869132,2251.293457
19252,0x6faff61b4781f481f2f9f627362a72905edd5534f6d3...,13947699,25,0x1c5a4f67124785c57b8f056fdc92c5b2e5877f09,0x3909336de913344701c6f096502d26208210b39f,WETH,USDC,3.311520,11381.427814,3609.706873,1.000261,11953.616276,11384.396454,569.219823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0x49ed18d5bffb7a18b194775d9f920142eb9fbf02f8ed...,16196235,46,0xb869561d958ecfaabed692b6c2b1ae37780fda13,0xc2a54f74ebbbba5ed92d6a0b5dcb0b0ffb96f36e,WETH,USDC,3.962309,4611.836409,1218.532638,0.997062,4828.202620,4598.288210,229.914410
66,0xb115135fc4e0409633cf3b9c7853cd3d11b0b6e97d59...,16196355,8,0xa20a0b22501e5507114ef1b0e88174f510f01ccb,0xb6569481dccddd527c2b0e8ba32f494e52224ca1,WETH,USDC,0.211112,245.718884,1215.661575,0.994713,256.640771,244.419782,12.220989
56,0x178a45df930f61856c8ab00685fd4317d408bc5221a2...,16200305,2,0xde092a220313cede58750434b66d46b5ff494cbb,0x6759674c937badbed743d56e5be86301968ea662,WETH,USDC,25.023853,28540.639743,1187.845764,0.991885,29724.478235,28309.026891,1415.451345
46,0x26cd3ae13c6afc36c798deeb8c15b2f4bd70e90e6e90...,16200454,57,0x70868f0fe4c17392eced8734be8bda110095d1cd,0x3444a6bb4030270b66117fa654aa60149a0212ed,WETH,USDC,8.186580,9146.191296,1170.430000,0.997742,9581.819261,9125.542153,456.277108


In [30]:
for i_ in range(1, 11):
    print(aave_liquidations_[(aave_liquidations_["borrowAsset"] == "USDC") & (aave_liquidations_["collateralAsset"] == "WETH")].txnHash.iloc[-i_])
    
    

0x0a92ef2318053d062d9b7c6033cf2c04494d38e586f4642ddfcc5bf6a5c1c03e
0x26cd3ae13c6afc36c798deeb8c15b2f4bd70e90e6e903b83e966d72fc5c1fd51
0x178a45df930f61856c8ab00685fd4317d408bc5221a2743148f200d94b009927
0xb115135fc4e0409633cf3b9c7853cd3d11b0b6e97d598bd25f18ed1891249374
0x49ed18d5bffb7a18b194775d9f920142eb9fbf02f8ed18052bf46b9ac27b1129
0x3ded80c142df689709438868e135043e8d8eb6a32f92c3247273f14287711d25
0x557d7c08aabd381dfcb9caddb0d35e30ddcd2ba634f3045a14ff4669411c13ba
0xe2f919cd792c5fae7d0ceb78bed5d555691f949f7a07c6079f98ae0d3b42c706
0x2f2af30783eb65850b1cea0a40ba78cdc7ad903c787a8c95184626272000b951
0xebcb67c390499c6ec01c31923316a0ecf1070a1d77361ff33ad73140eb26855f


In [19]:
aave_liquidations_["txnHash"].to_csv("./data/aave_txns.csv", index=False)

In [20]:
pd.read_csv("./data/aave_txns.csv")

,txnHash
0,0xa7a05399861e5d27df713654601283321629c4062734...
1,0x367f5f4cb77aaa3fb434d92faa2e95b7c06a1b347c36...
2,0xd9ef157cf44f884dd347d49857fce6701f5f49ec44d5...
3,0x9630822097eaf011583e30c4bc617222c07af85de75a...
4,0xab85193d071450cd95d68e9755131325bc27d5305489...
...,...
19270,0xf518fc2a8bab872919d25d7760957704e73f9731f2a7...
19271,0xbea7e992cb3be25fa340cd2e777fcaa6e14777d4049c...
19272,0xe0f1b5ba5b997a413ceff6f96d38c7df349cabf7c8a2...
19273,0xc26fb38db9e50752ef15b85de902415eaf6ca5577ae4...
